<a href="https://colab.research.google.com/github/obiedeh/QPSK-Wireless-Link-Simulator/blob/main/qpsk_modem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### qpsk_modem.py – Modem & pulse shaping

In [1]:
import numpy as np
from scipy.signal import upfirdn

def bits_to_qpsk(bits: np.ndarray) -> np.ndarray:
    """
    Map bits (0/1) to normalized QPSK symbols using Gray coding.
    Input length must be even.
    """
    bits = np.asarray(bits).astype(int)
    if bits.size % 2 != 0:
        raise ValueError("Number of bits must be even for QPSK mapping.")
    pairs = bits.reshape(-1, 2)
    mapping = {
        (0, 0): 1 + 1j,
        (0, 1): -1 + 1j,
        (1, 1): -1 - 1j,
        (1, 0): 1 - 1j,
    }
    syms = np.array([mapping[tuple(b)] for b in pairs], dtype=complex)
    return syms / np.sqrt(2)  # normalize to unit average power

def qpsk_to_bits(symbols: np.ndarray) -> np.ndarray:
    """
    Hard-decision QPSK demapper (normalized constellation).
    Returns an array of 0/1 bits.
    """
    symbols = np.asarray(symbols)
    bits_out = []
    for s in symbols:
        # Decide quadrant
        b0 = 0 if s.real >= 0 else 1
        b1 = 0 if s.imag >= 0 else 1
        # This matches the mapping inverse above
        if (b0, b1) == (0, 0):
            bits_out.extend([0, 0])
        elif (b0, b1) == (0, 1):
            bits_out.extend([0, 1])
        elif (b0, b1) == (1, 1):
            bits_out.extend([1, 1])
        else:  # (1, 0)
            bits_out.extend([1, 0])
    return np.array(bits_out, dtype=int)

def rrc_filter(num_taps: int, beta: float, sps: int) -> np.ndarray:
    """
    Generate a Root Raised Cosine (RRC) filter impulse response.

    num_taps: number of taps (odd is typical)
    beta: roll-off factor (0..1)
    sps: samples per symbol
    """
    if num_taps % 2 == 0:
        raise ValueError("num_taps should be odd for symmetric RRC filter.")
    t = np.arange(-num_taps // 2, num_taps // 2 + 1) / sps
    h = np.zeros_like(t, dtype=float)

    for i, ti in enumerate(t):
        if ti == 0.0:
            h[i] = 1.0 - beta + 4 * beta / np.pi
        elif beta != 0 and abs(ti) == 1 / (4 * beta):
            h[i] = (beta / np.sqrt(2)) * (
                ((1 + 2 / np.pi) * np.sin(np.pi / (4 * beta))) +
                ((1 - 2 / np.pi) * np.cos(np.pi / (4 * beta)))
            )
        else:
            num = (np.sin(np.pi * ti * (1 - beta)) +
                   4 * beta * ti * np.cos(np.pi * ti * (1 + beta)))
            den = (np.pi * ti * (1 - (4 * beta * ti) ** 2))
            h[i] = num / den if den != 0 else 0.0

    # Normalize energy
    h /= np.sqrt(np.sum(h ** 2))
    return h

def qpsk_modulate(bits: np.ndarray, sps: int = 8, beta: float = 0.35):
    """
    Full baseband QPSK modulation chain:
      bits -> symbols -> pulse shaping (RRC) -> upsampled baseband signal.

    Returns:
      tx_signal: shaped complex baseband signal
      syms: QPSK symbol sequence
      h_rrc: RRC filter used
    """
    bits = np.asarray(bits).astype(int)
    if bits.size % 2 != 0:
        bits = bits[:-1]  # drop last bit to keep it simple

    syms = bits_to_qpsk(bits)
    num_taps = 8 * sps + 1
    h_rrc = rrc_filter(num_taps, beta, sps)
    tx_signal = upfirdn(h_rrc, syms, up=sps)
    return tx_signal, syms, h_rrc

def qpsk_demodulate(rx_signal: np.ndarray,
                    h_rrc: np.ndarray,
                    sps: int,
                    num_syms: int,
                    channel_coef: complex = 1.0) -> np.ndarray:
    """
    Matched filter + downsample + equalize + hard-decision demap.

    Returns recovered bits.
    """
    # Matched filter (time-reversed conjugate)
    rx_filt = np.convolve(rx_signal, h_rrc[::-1].conj(), mode="same")

    # Symbol timing: assume perfect alignment at center
    offset = len(h_rrc) // 2
    rx_syms = rx_filt[offset::sps][:num_syms]

    # Equalize flat fading (scalar divide)
    rx_syms_eq = rx_syms / channel_coef

    bits_hat = qpsk_to_bits(rx_syms_eq)
    return bits_hat
